# La migration des érables à sucre au Canada

Ce document est divisé en quatre sections:
1. Lecture des données 
2. Création de la table de données de travail
3. Visualisation interactive des données 
4. Réponses aux questions de recherche

In [ ]:
import warnings
import geopandas
import rasterio
from pathlib import Path
import numpy as np
import pandas as pd
from rasterstats import zonal_stats
import matplotlib.pyplot as plt
from ipywidgets import interact
import matplotlib as mpl
from rasterio.plot import show
import plotly.io as pio
from rasterio.mask import mask
from shapely.geometry import Polygon

from utils import read_raster, reproject_raster, compute_area, zonal_stats_for_value, register_rasters, zonal_stats_intersection

pio.renderers.default = "notebook+pdf"
plt.rcParams['figure.figsize'] = [25, 15]
pd.set_option('display.max_columns', None, 'display.max_rows', None)

warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

## Définition des variables 

In [ ]:
DATA_PATH = Path("data/")
CRS = "EPSG:4326"

# 1. Lecture des données 

### 1.1. Données d'exploitations acéricoles et d'entailles d'érables

Les données d'exploitations et d'entailles proviennent de Statistique Canada. On télécharge les données pour [2011, 2016](https://www150.statcan.gc.ca/t1/tbl1/fr/cv!recreate.action?pid=3210042301&selectedNodeIds=&checkedLevels=0D1,0D2,0D3,0D4,0D5,1D1&refPeriods=20110101,20160101&dimensionLayouts=layout3,layout3,layout2&vectorDisplay=false) et [2021](https://www150.statcan.gc.ca/t1/tbl1/fr/cv.action?pid=3210036201), par région agricole de recensement (RAR).

In [ ]:
level = "RAR"

df_farms = pd.concat(
    [
        pd.read_csv(DATA_PATH / "fermes" / f"{level}_2021.csv", delimiter=";"), 
        pd.read_csv(DATA_PATH / "fermes" / f"{level}_2011_2016.csv", delimiter=";"), 
    ]
).drop(
    [
        "UNITÉ DE MESURE", 
        "IDENTIFICATEUR D'UNITÉ DE MESURE", 
        "FACTEUR SCALAIRE", 
        "VECTEUR", 
        "COORDONNÉES", 
        "IDENTIFICATEUR SCALAIRE", 
        "STATUS", 
        "SYMBOLE", 
        "TERMINÉ", 
        "DÉCIMALES"
    ], 
    axis=1
).reset_index()

df_farms["GÉO"] = df_farms.apply(
    lambda row: row["GÉO"].split(" [")[0], 
    axis=1
)

df_farms["DGUID"] = df_farms.apply(
    lambda row: row["DGUID"][4:], 
    axis=1
)


df_farms = df_farms.replace(
    "Nombre d'exploitations déclarantes", 
    "num_exploitations"
).replace(
    "Nombre d'entailles", 
    "num_entailles"
).replace(
    "Nombre de fermes déclarantes", 
    "num_exploitations"
).pivot(
    index=["GÉO", "DGUID"], 
    columns=["PÉRIODE DE RÉFÉRENCE", "Entailles d'érables"], 
    values="VALEUR"
)

df_farms.columns = [f"{x}_{y}" for x, y in df_farms.columns.to_flat_index()]

df_farms

On télécharge également les données par Division de Recensement (CD) ou Région Agricole de Recensement (RAR), selon la province. À noter que ces données ne sont utilisées que pour la visualisation des régions, et non pour l'analyse.

In [ ]:
def read_farms_df(level, provinces):
    farms = pd.read_csv(
        DATA_PATH / "fermes" / f"{level}_2021.csv", delimiter=";"
    ).drop(
        [
            "UNITÉ DE MESURE", 
            "IDENTIFICATEUR D'UNITÉ DE MESURE", 
            "FACTEUR SCALAIRE", 
            "VECTEUR", 
            "COORDONNÉES", 
            "IDENTIFICATEUR SCALAIRE", 
            "STATUS", 
            "SYMBOLE", 
            "TERMINÉ", 
            "DÉCIMALES"
        ], 
        axis=1
    ).reset_index()

    farms["GÉO"] = farms.apply(
        lambda row: row["GÉO"].split(" [")[0], 
        axis=1
    )

    farms["DGUID"] = farms.apply(
        lambda row: row["DGUID"][4:], 
        axis=1
    )

    farms["Province"] = farms.apply(
        lambda row: row["GÉO"].split(", ")[-1], 
        axis=1
    )

    farms = farms[farms["Province"].isin(provinces)]

    farms = farms.replace(
        "Nombre d'exploitations déclarantes", 
        "num_exploitations"
    ).replace(
        "Nombre d'entailles", 
        "num_entailles"
    ).replace(
        "Nombre de fermes déclarantes", 
        "num_exploitations"
    ).pivot(
        index=["GÉO", "DGUID"], 
        columns=["PÉRIODE DE RÉFÉRENCE", "Entailles d'érables"], 
        values="VALEUR"
    )

    farms.columns = [f"{x}_{y}" for x, y in farms.columns.to_flat_index()]
    farms["level"] = level
    
    return farms


df_farms_RAR = read_farms_df(
    "RAR", 
    provinces=[
        "Québec", 
        "Terre-Neuve-et-Labrador", 
        "Manitoba", 
        "Saskatchewan",  
        "Alberta",    
        "Colombie-Britannique",
])
df_farms_CD = read_farms_df(
    "CD", 
    provinces=[
        "Ontario", 
        "Nouveau-Brunswick",
        "Nouvelle-Écosse", 
        "Île-du-Prince-Édouard", 
])

df_farms_CD

### 1.2. Fichiers des limites géographiques

On télécharge également les fichiers des [limites géographiques des régions agricoles de recensement (RAR)](https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/boundary-limites/index2021-fra.cfm?year=21) à partir du site de Statistique Canada.

In [ ]:
df_boundaries = geopandas.read_file(
    DATA_PATH / "boundaries" / "lcar000b21a_e.shp"
).to_crs(
    CRS
).drop(
    ["CARUID", "CARENAME", "LANDAREA"], 
    axis=1
).rename(
    columns={"CARFNAME": "area_name"}
)

df_boundaries["DGUID"] = df_boundaries.apply(
    lambda row: row["DGUID"][4:], 
    axis=1
)

df_boundaries["total_area"] = compute_area(df_boundaries)

df_boundaries

On visualise les limites géographiques des Régions Agricoles de Recensement.

In [ ]:
df_boundaries.plot()

On télécharge également les fichiers des [limites géographiques des divisions de recensement (CD)](https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/boundary-limites/index2021-fra.cfm?year=21) à partir du site de Statistique Canada.

In [ ]:
df_boundaries_CD = geopandas.read_file(
    DATA_PATH / "boundaries" / "ldr_000b21a_f.shp"
).to_crs(
    CRS
).drop(
    ["DRIDU", "DRGENRE", "SUPTERRE"], 
    axis=1
).rename(
    columns={
        "DRNOM": "area_name", 
        "IDUGD": "DGUID", 
        "PRIDU": "PRUID"
    }
)

df_boundaries_CD["DGUID"] = df_boundaries_CD.apply(
    lambda row: row["DGUID"][4:], 
    axis=1
)

df_boundaries_CD

On visualise les limites géographiques des Divisions de Recensement.

In [ ]:
df_boundaries_CD.plot()

### 1.3. Projections de l'enveloppe climatique propice à l'érable à sucre 

On télécharge ensuite les fichiers de projections climatiques pour l'érable à sucre et l’érable rouge (selon le scénario SDM ANUCLIM et le RCP 8.5) pour les horizons 2011-2040, 2041-2070 et 2071-2100.

On considère une enveloppe "marginale" et une enveloppe "core", selon les définitions suivantes :
- "marginale" : enveloppe climatique pour une période définie calibrée selon le min/max des conditions climatiques observées durant la période de référence.
- "core" : Enveloppe climatique selon le 5e à 95e centile des conditions climatiques observées durant la période de référence 

_Source : https://protect-eu.mimecast.com/s/G3lTCVNkkFqzB6DHGqvem?domain=planthardiness.gc.ca ET McKENNEY, D. W., Pedlar, J. H., Rood, R. B., & Price, D. (2011). Revisiting projected shifts in the climate envelopes of North American trees using updated general circulation models. Global Change Biology, 17(8), 2720-2730._

#### 1.3.1. Projection 2011-2040

In [ ]:
filename = DATA_PATH / "rasters" / "arcp8510000532011-2040.asc"

raster_2011_2040 = read_raster(filename, crs=CRS)
    
cmap = mpl.colors.ListedColormap(['black', 'yellow', 'orange'])
bounds = [raster_2011_2040.nodata, 1, 3, 4]
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

raster_2011_2040_data = raster_2011_2040.read(1)
plt.imshow(raster_2011_2040_data, cmap=cmap, norm=norm, alpha=0.5)
plt.show()

Pour chacune des régions agricoles de recensement, on calcule le nombre de pixels pour chacune des catégories ("marginale", "core" et "autres").

In [ ]:
period = "2011-2040"

for value, name in [
    (1, "marginal"), 
    (3, "core"), 
    (raster_2011_2040.nodata, "rest")
]:
    df_boundaries[f"{name}_{period}"] = zonal_stats_for_value(
        raster=raster_2011_2040, 
        vectors=df_boundaries["geometry"], 
        value=value, 
        data_value=100, 
        stats="count", 
        resolution=2.5
    )

#### 1.3.2. Projection 2041-2070

In [ ]:
filename = DATA_PATH / "rasters" / "arcp8510000532041-2070.asc"

raster_2041_2070 = read_raster(filename, crs=CRS)

raster_2041_2070_data = raster_2041_2070.read(1)
plt.imshow(raster_2041_2070_data, cmap=cmap, norm=norm, alpha=0.5)
plt.show()

Pour chacune des régions agricoles de recensement, on calcule le nombre de pixels pour chacune des catégories ("marginale", "core" et "autres").

In [ ]:
period = "2041-2070"

for value, name in [
    (1, "marginal"), 
    (3, "core"), 
    (raster_2041_2070.nodata, "rest")
]:
    df_boundaries[f"{name}_{period}"] = zonal_stats_for_value(
        raster=raster_2041_2070, 
        vectors=df_boundaries["geometry"], 
        value=value, 
        data_value=100, 
        stats="count", 
        resolution=2.5
    )

#### 1.3.3. Projection 2071-2100

In [ ]:
filename = DATA_PATH / "rasters" / "arcp8510000532071-2100.asc"
raster_2071_2100 = read_raster(filename, crs=CRS)

raster_2071_2100_data = raster_2071_2100.read(1)
plt.imshow(raster_2071_2100_data, cmap=cmap, norm=norm, alpha=0.5)
plt.show()

Pour chacune des régions agricoles de recensement, on calcule le nombre de pixels pour chacune des catégories ("marginale", "core" et "autres").

In [ ]:
period = "2071-2100"

for value, name in [
    (1, "marginal"), 
    (3, "core"), 
    (raster_2071_2100.nodata, "rest")
]:
    df_boundaries[f"{name}_{period}"] = zonal_stats_for_value(
        raster=raster_2071_2100, 
        vectors=df_boundaries["geometry"], 
        value=value, 
        data_value=100, 
        stats="count", 
        resolution=2.5
    )

### 1.4. Intersection des projections

Pour chacune des régions, on observe la proportion qui demeure intacte d'une période à une autre.

In [ ]:
rasters, transform = register_rasters(
    raster_2011_2040, 
    raster_2041_2070, 
    raster_2071_2100
)

registered_raster_2011_2040, registered_raster_2041_2070, registered_raster_2071_2100 = rasters

In [ ]:
stats, marginal_2011_and_2041_raster = zonal_stats_intersection(
    df_boundaries['geometry'], 
    registered_raster_2011_2040, 
    registered_raster_2041_2070, 
    affine=transform, 
    data_values=[1, 3], 
    nodata_value=-999, 
    resolution=2.5
)
df_boundaries[f"marginal_2011_and_2041"] = stats

stats, core_2011_and_2041_raster = zonal_stats_intersection(
    df_boundaries['geometry'], 
    registered_raster_2011_2040, 
    registered_raster_2041_2070, 
    affine=transform, 
    data_values=[3], 
    nodata_value=-999, 
    resolution=2.5
)
df_boundaries[f"core_2011_and_2041"] = stats
    
stats, marginal_2011_and_2071_raster = zonal_stats_intersection(
    df_boundaries['geometry'], 
    registered_raster_2011_2040, 
    registered_raster_2071_2100, 
    affine=transform, 
    data_values=[1, 3], 
    nodata_value=-999, 
    resolution=2.5
)
df_boundaries[f"marginal_2011_and_2071"] = stats

stats, core_2011_and_2071_raster = zonal_stats_intersection(
    df_boundaries['geometry'], 
    registered_raster_2011_2040, 
    registered_raster_2071_2100, 
    affine=transform, 
    data_values=[3], 
    nodata_value=-999, 
    resolution=2.5
)
df_boundaries[f"core_2011_and_2071"] = stats

# 2. Création de la table de données de travail

On joint les données téléchargées dans un dataframe comprenant toutes les informations requises pour répondre aux questions de recherche. On calcule quelques statistiques préalables et on nettoie la table de données finale.

In [ ]:
df = pd.merge(
    df_boundaries, 
    df_farms, 
    on="DGUID"
)

pr_df = pd.DataFrame({
    "PRUID": ["10", "11", "12", "13", "24", "35", "46", "47", "48", "59", "60", "61", "62"], 
    "province": [
        "Terre-Neuve-et-Labrador", 
        "Île-du-Prince-Édouard", 
        "Nouvelle-Écosse",
        "Nouveau-Brunswick",
        "Québec",
        "Ontario",
        "Manitoba",
        "Saskatchewan",
        "Alberta",
        "Colombie-Britannique",
        "Yukon",
        "Territoires du Nord-Ouest",
        "Nunavut"
    ]
}
)

df = df.merge(
    pr_df, 
    on="PRUID"
).drop("PRUID", axis=1)

df["rank_entailles"] = df["2021_num_entailles"].rank(method="min", na_option="bottom", ascending=False)
df["rank_exploitations"] = df["2021_num_exploitations"].rank(method="min", na_option="bottom", ascending=False)

df = df.rename({
    "all_2011-2040": "rest_2011-2040", 
    "all_2041-2070": "rest_2041-2070", 
    "all_2071-2100": "rest_2071-2100", 
}, axis=1)

df

On crée également une table de données avec différents niveaux de délimitation des régions (CD ou RAR) selon la province. Ces données ne seront utilisées que pour la visualisation.

In [ ]:
df_CD_and_RAR = geopandas.GeoDataFrame(
    pd.concat([
        pd.merge(
            df_farms_CD,
            df_boundaries_CD,  
            on="DGUID", 
            how="left"
        ),  
        pd.merge(
            df_farms_RAR,
            df_boundaries,  
            on="DGUID", 
            how="left"
        )
    ])
)

df_CD_and_RAR

# 3. Visualisation des données 

## 3.1. Projection des niches climatiques

On visualise la distribution des entailles en 2021 par rapport à la carte des niches climatiques.

La niche climatique marginale est en jaune et la niche climatique core, en orange. Les cercles bleus pointent vers le centroïde de chacune des régions agricoles de recensement et la taille du cercle est proportionnelle au nombre d'entailles en 2021.

### 3.1.1. Projection 2011-2040

In [ ]:
def _plot(df, raster, markersize_factor=50000, extent=None):
    fig, ax = plt.subplots()

    ax = show(
        raster, 
        extent=[
            raster.bounds[0], 
            raster.bounds[2], 
            raster.bounds[1], 
            raster.bounds[3]
        ], 
        alpha=0.5,
        cmap=cmap, 
        norm=norm,
        with_bounds=True, 
        ax=ax
    )

    df.boundary.plot(ax=ax, lw=0.15, color="black")
    df.centroid.plot(
        markersize=df['2021_num_entailles'] / markersize_factor, 
        alpha=0.5,
        color="DarkTurquoise", 
        ax=ax
    )
    
_plot(df_CD_and_RAR, raster_2011_2040)

### 3.1.2. Projection 2041-2070

In [ ]:
_plot(df_CD_and_RAR, raster_2041_2070)

### 3.1.3. Projection 2071-2100

In [ ]:
_plot(df_CD_and_RAR, raster_2071_2100)

On zoom ici sur l'Est du pays, où se concentre la production acéricole.

### 3.1.4. Projection 2011-2040

In [ ]:
def _plot_zoom(df, raster, markersize_factor=5000):
    xmin = -90
    xmax = -40
    ymin = 42
    ymax = 55
    
    crop_df = df.cx[xmin: xmax, ymin: ymax]
    
    xmin = min(crop_df.bounds["minx"])
    ymin = min(crop_df.bounds["miny"])
    xmax = max(crop_df.bounds["maxx"])
    ymax = max(crop_df.bounds["maxy"])
    
    fig, ax = plt.subplots()
    
    out_image, out_transform = mask(
        raster, 
        [Polygon(((xmin, ymin), (xmax, ymin) ,(xmax, ymax), (xmin, ymax),  (xmin, ymin)))], 
        crop=True
    )

    ax = show(
        out_image, 
        transform=out_transform, 
        alpha=0.5,
        cmap=cmap, 
        norm=norm,
        with_bounds=True, 
        ax=ax
    )

    crop_df.boundary.plot(ax=ax, lw=0.75, color="black")
    
    centroids = crop_df.centroid
    centroids.plot(
        markersize=crop_df['2021_num_entailles'] / markersize_factor, 
        alpha=0.35,
        color="DarkTurquoise", 
        ax=ax, 
    )
    centroids.plot(
        markersize=3,
        alpha=1,
        color="DarkSlateGrey", 
        ax=ax, 
    )

    ax.set_axis_off()
    
_plot_zoom(df_CD_and_RAR, raster_2011_2040)

### 3.1.5. Projection 2041-2070

In [ ]:
_plot_zoom(df_CD_and_RAR, raster_2041_2070)

### 3.1.6. Projection 2071-2100

In [ ]:
_plot_zoom(df_CD_and_RAR, raster_2071_2100)

## 3.2. Déplacement des niches climatiques

On visualise maintenant les régions qui perdent les conditions favorables au développement de l'érable à sucre. Plus précisément, on observe en rouge les régions qui faisaient partie de la niche climatique dans la période de projection initiale (2011-2040), mais qui n'en font plus partie dans la période de projection finale.

### 3.2.1. Conditions cores seulement : 2011-2040 à 2041-2070

In [ ]:
def _plot_lost_terrain(df, raster1, raster2, extent, data_value, nodata_value=-999, markersize_factor=50000):
    new_raster = np.ones(raster1.shape) * nodata_value
    new_raster[np.where((raster1 >= data_value) & (raster2 < data_value))] = 1
    
    fig, ax = plt.subplots()

    cmap = mpl.colors.ListedColormap(['black', 'red'])
    bounds = [nodata_value, 1, 3]
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

    
    ax = show(
        new_raster,
        extent=extent,
        alpha=0.5,
        cmap=cmap, 
        norm=norm,
        with_bounds=True, 
        ax=ax
    )

    df.boundary.plot(ax=ax, lw=0.15, color="black")
    df.centroid.plot(
        markersize=df['2021_num_entailles'] / markersize_factor, 
        alpha=0.5,
        color="DarkTurquoise", 
        ax=ax
    )

_plot_lost_terrain(
    df_CD_and_RAR,
    registered_raster_2011_2040, 
    registered_raster_2041_2070, 
    extent=[
        raster_2011_2040.bounds[0], 
        raster_2011_2040.bounds[2], 
        raster_2011_2040.bounds[1], 
        raster_2011_2040.bounds[3]
    ], 
    data_value=3, 
)

### 3.2.2. Conditions cores seulement : 2011-2040 à 2071-2100

In [ ]:
_plot_lost_terrain(
    df_CD_and_RAR, 
    registered_raster_2011_2040, 
    registered_raster_2071_2100, 
    extent=[
        raster_2011_2040.bounds[0], 
        raster_2011_2040.bounds[2], 
        raster_2011_2040.bounds[1], 
        raster_2011_2040.bounds[3]
    ], 
    data_value=3, 
)

### 3.2.3. Conditions cores et marginales : 2011-2040 vs 2041-2070

In [ ]:
_plot_lost_terrain(
    df_CD_and_RAR, 
    registered_raster_2011_2040, 
    registered_raster_2041_2070, 
    extent=[
        raster_2011_2040.bounds[0], 
        raster_2011_2040.bounds[2], 
        raster_2011_2040.bounds[1], 
        raster_2011_2040.bounds[3]
    ], 
    data_value=1, 
)

### 3.2.4. Conditions cores et marginales : 2011-2040 vs 2071-2100

In [ ]:
_plot_lost_terrain(
    df_CD_and_RAR, 
    registered_raster_2011_2040, 
    registered_raster_2071_2100, 
    extent=[
        raster_2011_2040.bounds[0], 
        raster_2011_2040.bounds[2], 
        raster_2011_2040.bounds[1], 
        raster_2011_2040.bounds[3]
    ], 
    data_value=1, 
)

# 4. Réponses aux questions de recherche

Pour répondre aux questions de recherche, on définit deux scénarios :
- Scénario A : on ne considère que les zones avec les conditions climatiques cores.
- Scénario B : on considère les zones avec les conditions climatiques cores et marginales.

On décline également la réponse par rapport aux projections de 2041-2070, puis aux projections de 2071-2100.

On s'attend à ce que les différences les plus marquées soient observées selon le scénario A, avec les projections de 2071-2100, et les différences les moins marquées avec le scénario B et les projections de 2041-2070.

Les conclusions aux questions de recherche sont affichées <span style="color:red"> en rouge </span>.

## 4.1. Où se situent les zones propices au développement de l’érable à sucre au Canada et comment seront-elles affectées par le réchauffement climatique?

### 4.1.1.  Y a-t-il des zones actuellement propices qui ne le seront plus du tout?

In [ ]:
# Scénario A 
df["diff_A_2011_vs_2041"] = df["core_2011-2040"] - df["core_2011_and_2041"]
df["diff_A_2011_vs_2071"] = df["core_2011-2040"] - df["core_2011_and_2071"]

# Scénario B
df["core_marginal_2011-2040"] = df["core_2011-2040"] + df["marginal_2011-2040"]
df["core_marginal_2041-2070"] = df["core_2041-2070"] + df["marginal_2041-2070"]
df["core_marginal_2071-2100"] = df["core_2071-2100"] + df["marginal_2071-2100"]
df["diff_B_2011_vs_2041"] = df["core_marginal_2011-2040"] - df["marginal_2011_and_2041"]
df["diff_B_2011_vs_2071"] = df["core_marginal_2011-2040"] - df["marginal_2011_and_2071"]

#### Scénario A (2011-2040 vs 2041-2070)

On affiche les données ordonnées en fonction des pertes de zones idéales (`diff_A_2011_vs_2041`). 
- `core_2011-2040` et `core_2041-2070` représentent l'aire totale (en kilomètres carrés) des zones idéales pour les périodes de projection.
- `core_2011_and_2041` représente l'aire totale idéale en 2011 et qui le serait toujours en 2041.

<span style="color:red"> 
On constate que les régions du Nord, du Sud et de l'Est de l'Ontario sont On constate que le Nord, le Sud et l'Est de l'Ontario sont les régions où on enregistrerait les plus importantes pertes de zones idéales. Malgré tout, les projections de 2041-2070 indiquent qu'une grande partie de ces régions serait encore idéale au développement de l'érable à sucre (19 750 km carrés pour le Nord, 669 km carrés pour le Sud et 294 km carrés pour l'Est). De cette aire en 2041, une bonne partie est déjà dans des conditions optimales dans la période actuelle de projection (13 000 km carrés pour le Nord, 600 km carrés pour le Sud et 294 km carrés pour l'Est).
<br/>
<br/>
Il en va de même pour la région agricole de recensement 2 de Terre-Neuve-et-Labrador où 1788 km carrés demeureraient dans la niche climatique core de l’érable, la quasi-totalité de cette aire s’y trouvant également dans la période actuelle de projection.
    
À l'inverse, les régions agricoles de recensement 2 et 3 du Nouveau-Brunswick ainsi que la Montérégie (Québec) enregistreraient d'importantes pertes. On constate que ces régions se retrouveraient dans de nouvelles conditions de croissance pour l'érable à sucre en 2041 (moins de 56 km carrés demeureraient dans la niche climatique core).
    
De même, la région agricole de recensement 3 de la Nouvelle-Écosse ainsi que les régions agricoles de recensement 2 et 3 de l'Île-du-Prince-Édouard sont actuellement dans la niche climatique core, mais ne seraient plus du tout (moins de 10 km carrés) en 2041.
</span>

In [ ]:
df.sort_values(
    "diff_A_2011_vs_2041", 
    ascending=False
)[[
    "area_name", 
    "province", 
    "diff_A_2011_vs_2041", 
    "core_2011-2040",
    "core_2041-2070",
    "core_2011_and_2041"
]].reset_index()

<span style="color:red"> 
En outre, ces régions, qui se trouvaient aussi dans la niche climatique core en 2011-2040, en sortiraient en 2041-2070.
</span>

In [ ]:
df[
    (df["core_2011-2040"] > 0) & 
    (df["core_2041-2070"] == 0) & 
    (df["core_2011_and_2041"] == 0)
].sort_values("core_2011-2040")[[
    "area_name", 
    "province",
    "core_2011-2040", 
    "core_2041-2070",
    "core_2011_and_2041"
]]

#### Scénario A (2011-2040 vs 2071-2100)

On affiche les données ordonnées en fonction des pertes de zones idéales (`diff_A_2011_vs_2071`). 
- `core_2011-2040` et `core_2041-2100` représentent l'aire totale (en kilomètres carrés) des zones idéales pour les périodes de projection.
- `core_2011_and_2071` représente l'aire totale idéale en 2011 et qui le serait toujours en 2071.

<span style="color:red"> 
Les conclusions sont plus dramatiques en 2071-2100. Presque toutes les régions d’un bout à l’autre de pays, à l’exception du nord du Québec, sortiraient de la niche climatique core de l’érable. 
<br/>
<br/> 
On note quelques exceptions, notamment la Mauricie (Québec) et la région agricole de recensement 2 de Terre-Neuve-et-Labrador, où une faible partie du territoire demeurerait dans des conditions de croissance idéales pour l'érable à sucre (187 km carrés pour la Mauricie et 337 km carrés pour la Région agricole de recensement 2 de T.-N.-L.).
<br/>
<br/>
Les régions de l'Abitibi-Témiscamingue et du Saguenay-Lac-Saint-Jean-Côte-Nord feraient également exception. Ces régions enregistreraient parmi les pertes les plus importantes au pays, mais verraient un accroissement de l’aire idéale au sein même de leur territoire (54 000 km carrés pour l'Abitibi et 26 000 km carrés pour le Saguenay). Il s'agit toutefois de nouvelles zones, c'est-à-dire que les zones qui seraient idéales en 2071 ne le sont pas dans la période actuelle de projection.
</span>

In [ ]:
df.sort_values(
    "diff_A_2011_vs_2071", 
    ascending=False
).reset_index()[[
    "area_name",  
    "province",
    "diff_A_2011_vs_2071", 
    "core_2011-2040",
    "core_2071-2100",
    "core_2011_and_2071",
]]

<span style="color:red"> 
Ces régions se trouvaient en partie dans des conditions idéales en 2011-2040, mais en sortiraient complètement en 2071-2100.
</span>

In [ ]:
df[
    (df["core_2011-2040"] > 0) & 
    (df["core_2071-2100"] == 0) & 
    (df["core_2011_and_2071"] == 0)
].sort_values("core_2011-2040", ascending=False)[[
    "area_name", 
    "province",
    "core_2011-2040", 
    "core_2071-2100", 
    "core_2011_and_2071"
]]

#### Scénario B (2011-2040 vs 2041-2070)

On affiche les données ordonnées en fonction des pertes de zones idéales et tolérables (`diff_B_2011_vs_2041`). 
- `core_marginal_2011-2040` et `core_marginal_2041-2070` représentent l'aire totale (en kilomètre carrés) des zones idéales et tolérables pour les périodes de projection.
- `marginal_2011_and_2041` représente l'aire totale aux conditions idéales et tolérables en 2011-2040 et qui le serait toujours en 2041-2070. 

<span style="color:red"> 
Lorsqu'on considère les conditions climatiques core et marginale, on observe les pertes les plus importantes dans des régions où l'aire propice serait plus grande en 2041-2070 qu'elle ne l'était en 2011-2040. On observe donc clairement un déplacement des aires au sein même de ces régions. On note entre autres la région du Nord de l'Ontario, qui enregistrerait une perte de 20 000 km carrés d'aire propice, mais qui verrait la création de plus 33 000 km carrés de nouvelle aire propice.
</span>

In [ ]:
df.sort_values(
    "diff_B_2011_vs_2041", 
    ascending=False
).reset_index()[[
    "area_name",  
    "province",
    "diff_B_2011_vs_2041", 
    "core_marginal_2011-2040", 
    "core_marginal_2041-2070",
    "marginal_2011_and_2041"
]]

<span style="color:red"> 
Ces régions se trouvaient dans des conditions de croissance idéales ou tolérables en 2011-2040, mais en sortiraient complètement en 2041-2070.
</span>

In [ ]:
df[
    (df["core_marginal_2011-2040"] > 0) & 
    (df["core_marginal_2041-2070"] == 0) & 
    (df["marginal_2011_and_2041"] == 0)
].sort_values("core_2011-2040", ascending=False)[[
    "area_name", 
    "province",
    "core_marginal_2011-2040", 
    "core_marginal_2041-2070", 
    "marginal_2011_and_2041"
]]

#### Scénario B (2011-2040 vs 2071-2100)


On affiche les données ordonnées en fonction des pertes de zones idéales et tolérables (`diff_B_2011_vs_2071`). 
- `core_marginal_2011-2040` et `core_marginal_2071-2100` représentent l'aire totale (en kilomètre carrés) des zones idéales ou tolérables pour les périodes de projection.
- `marginal_2011_and_2071` représente l'aire totale aux conditions idéales et tolérables en 2011 et qui le serait toujours en 2071.

<span style="color:red"> 
En 2071-2100, le portrait est similaire.
<br/>
<br/>
On note toutefois au 8e rang des pertes de zones propices, la région de l'Outaouais. L'entièreté des zones qui se trouvaient dans des conditions idéales ou tolérables dans la période actuelle de projection ne le serait plus en 2071-2100. Il en va de même pour la région agricole de recensement du Nouveau-Brunswick, le Bas-Saint-Laurent et les Laurentides, où moins de 100 km carrés de la zone actuelle resteraient propices en 2071-2100.
</span>

In [ ]:
df.sort_values(
    "diff_B_2011_vs_2071", 
    ascending=False
).reset_index()[[
    "area_name",  
    "province",
    "diff_B_2011_vs_2071", 
    "core_marginal_2011-2040", 
    "core_marginal_2071-2100",
    "marginal_2011_and_2071",
]]

<span style="color:red"> 
Ces régions se trouvaient en partie dans la niche climatique core et marginale de l’érable en 2011-2040, mais en sortiraient complètement en 2071-2100.
</span>

In [ ]:
df[
    (df["core_marginal_2011-2040"] > 0) & 
    (df["core_marginal_2071-2100"] == 0) & 
    (df["marginal_2011_and_2071"] == 0)
].sort_values("core_2011-2040", ascending=False)[[
    "area_name", 
    "province",
    "core_marginal_2011-2040", 
    "core_marginal_2071-2100", 
    "marginal_2011_and_2071"
]]

### 4.1.2. Quelle proportion du Canada et des provinces est actuellement favorable au développement de l’érable à sucre? Qu'elle deviendra cette proportion avec le réchauffement climatique?

#### Canada 

On calcule la proportion du Canada qui se trouve dans la niche climatique de l'érable en 2011-2040, 2041-2070 et 2071-2100, selon les scénarios A et B.

<span style="color:red"> 
On constate que la proportion du territoire canadien propice au développement de l'érable pourrait s'accroître avec le temps, qu'on se fit au scénario A ou B. En effet, l'enveloppe climatique ne rétrécirait pas, mais elle se déplacerait plutôt vers le Nord.
</span>

In [ ]:
sum_A_2011 = df['core_2011-2040'].sum() / (df['rest_2011-2040'].sum() + df['core_marginal_2011-2040'].sum())
sum_B_2011 = df['core_marginal_2011-2040'].sum() / (df['rest_2011-2040'].sum() + df['core_marginal_2011-2040'].sum())

sum_A_2041 = df['core_2041-2070'].sum() / (df['rest_2041-2070'].sum() + df['core_marginal_2041-2070'].sum())
sum_B_2041 = df['core_marginal_2041-2070'].sum() / (df['rest_2041-2070'].sum() + df['core_marginal_2041-2070'].sum())

sum_A_2071 = df['core_2071-2100'].sum() / (df['rest_2071-2100'].sum() + df['core_marginal_2071-2100'].sum())
sum_B_2071 = df['core_marginal_2071-2100'].sum() / (df['rest_2071-2100'].sum() + df['core_marginal_2071-2100'].sum())

print(f"Scénario A : {round(sum_A_2011, 3)} (2011), {round(sum_A_2041, 3)} (2041), {round(sum_A_2071, 3)} (2071)")
print(f"Scénario B : {round(sum_B_2011, 3)} (2011), {round(sum_B_2041, 3)} (2041), {round(sum_B_2071, 3)} (2071)")

<span style="color:red"> 
Selon le scénario A, c'est 70 % de l'aire idéale actuelle qui le serait toujours en 2041-2070, contre seulement 2 % en 2071-2100.
<br/>
Selon le scénario B, c'est 81 % de l'aire idéale et tolérable actuelle qui le serait toujours en 2041-2070, contre seulement 65 % en 2071-2100. 
</span>

In [ ]:
sum_A_2041 = df['core_2011_and_2041'].sum() / df['core_2011-2040'].sum()
sum_B_2041 = df['marginal_2011_and_2041'].sum() / df['core_marginal_2011-2040'].sum()

sum_A_2071 = df['core_2011_and_2071'].sum() / df['core_2011-2040'].sum()
sum_B_2071 = df['marginal_2011_and_2071'].sum() / df['core_marginal_2011-2040'].sum()

print(f"Scénario A : {round(sum_A_2041, 3)} (2041), {round(sum_A_2071, 3)} (2071)")
print(f"Scénario B : {round(sum_B_2041, 3)} (2041), {round(sum_B_2071, 3)} (2071)")

#### Provinces

On calcule ensuite ces mêmes proportions à l'échelle des provinces. 
- `sum_X_YYYY` représente la proportion pour l'année de départ d'une projection donnée (`YYYY`) et un scénario (`X`).

<span style="color:red"> 
On observe un recul important de la proportion de la niche climatique de l’érable dans les provinces du Nouveau-Brunswick, de l'Ontario et de l'Île-du-Prince-Édouard. En 2071-2100, l'entièreté de ces trois provinces se trouverait dans de nouvelles conditions de croissance (selon le scénario A). La proportion de zones idéales ou favorables demeure plus stable dans le temps en considérant plutôt le scénario B.
<br/>
<br/>
Au Québec, une plus grande portion de la province serait propice à la croissance de l’érable avec le temps, tant selon le scénario A que le scénario B. Il en va de même pour Terre-Neuve-et-Labrador, province qui est présentement exclue dans la niche climatique de l’érable.
</span>

In [ ]:
df_pr = df.groupby(
    "province"
).sum()

df_pr["sum_A_2011"] = df_pr['core_2011-2040'] / (df_pr['rest_2011-2040'] + df_pr['core_marginal_2011-2040'])
df_pr["sum_B_2011"] = df_pr['core_marginal_2011-2040'] / (df_pr['rest_2011-2040'] + df_pr['core_marginal_2011-2040'])

df_pr["sum_A_2041"] = df_pr['core_2041-2070'] / (df_pr['rest_2041-2070'] + df_pr['core_marginal_2041-2070'])
df_pr["sum_B_2041"] = df_pr['core_marginal_2041-2070'] / (df_pr['rest_2041-2070'] + df_pr['core_marginal_2041-2070'])

df_pr["sum_A_2071"] = df_pr['core_2071-2100'] / (df_pr['rest_2071-2100'] + df_pr['core_marginal_2071-2100'])
df_pr["sum_B_2071"] = df_pr['core_marginal_2071-2100'] / (df_pr['rest_2071-2100'] + df_pr['core_marginal_2071-2100'])

df_pr[[
    "sum_A_2011", 
    "sum_A_2041", 
    "sum_A_2071", 
    "sum_B_2011", 
    "sum_B_2041", 
    "sum_B_2071", 
]]

<span style="color:red"> 
Selon le scénario A, la proportion des aires avec des conditions actuellement idéales, et qui le demeureraient en 2041-2070, est de 92 % pour le Québec, contre seulement 61 % pour T.-N.-L., 52 % pour le N.-B., 50 % pour l'Ontario, 10 % pour l'Î.-P.-É. et 0 % pour la N.-É. En 2071-2100, cette proportion s'effriterait : il ne resterait que 13 % du territoire propice à Terre-Neuve-et-Labrador et à peine 3 % au Québec. Pour toutes les autres provinces, la proportion d'aires avec des conditions actuellement idéales, et qui le seraient toujours en 2071-2100, est nulle.
</span>

In [ ]:
df_pr = df.groupby(
    "province"
).sum()


df_pr["sum_A_2041"] = df_pr['core_2011_and_2041'] / df_pr['core_2011-2040']
df_pr["sum_B_2041"] = df_pr['marginal_2011_and_2041'] / df_pr['core_marginal_2011-2040']

df_pr["sum_A_2071"] = df_pr['core_2011_and_2071'] / df_pr['core_2011-2040']
df_pr["sum_B_2071"] = df_pr['marginal_2011_and_2071'] / df_pr['core_marginal_2011-2040']

df_pr[[
    "sum_A_2041",
    "sum_A_2071", 
    "sum_B_2041", 
    "sum_B_2071", 
]]

## 4.2. Où se concentre la production de sirop d’érable au Canada. Et comment a-t-elle évolué au fil des années?

### 4.2.1. Où se concentre la production de sirop d’érable au Canada?

#### Proportions par région agricole de recensement

On affiche les données ordonnées en fonction du nombre d'entailles en 2021 (`2021_num_entailles`). On affiche également :
- le nombre d'exploitations en 2021 (`2021_num_exploitations`)
- le rang de la région agricole de recensement en fonction du nombre d'entailles (`rank_entailles`) et du nombre d'exploitations acéricoles (`rank_exploitations`).

<span style="color:red"> 
Le Québec domine l'exploitation des érables au Canada : 32 % des entailles sont dans la région de Chaudière-Appalaches, 15% en Estrie et 15% au Bas-Saint-Laurent. On retrouve également une importante production au Nouveau-Brunswick, en Ontario et, dans une moindre mesure, en Nouvelle-Écosse.
</span>

In [ ]:
num_entailles_2021 = df["2021_num_entailles"].sum()
num_entailles_2016 = df["2016_num_entailles"].sum()
num_entailles_2011 = df["2011_num_entailles"].sum()

num_exploitations_2021 = df["2021_num_exploitations"].sum()
num_exploitations_2016 = df["2016_num_exploitations"].sum()
num_exploitations_2011 = df["2011_num_exploitations"].sum()

df["2021_percent_entailles"] = df["2021_num_entailles"] / num_entailles_2021
df["2021_percent_exploitations"] = df["2021_num_exploitations"] / num_exploitations_2021

df.sort_values(
    "2021_num_entailles", 
    ascending=False
).reset_index()[[
    "area_name", 
    "province", 
    "2021_num_entailles", 
    "2021_percent_entailles", 
    "2021_num_exploitations", 
    "2021_percent_exploitations", 
    "rank_entailles",
    "rank_exploitations"
]]

#### Proportions d'entailles par province 

<span style="color:red"> 
Au total, 90 % des entailles sont au Québec. Le Nouveau-Brunswick suit avec 6 % des entailles en 2021, puis l'Ontario avec 3 %.
</span>

In [ ]:
df_pr = df.groupby(
    "province"
).sum()

df_pr["2021_percent_entailles"] = df_pr["2021_num_entailles"] / num_entailles_2021
df_pr["2016_percent_entailles"] = df_pr["2016_num_entailles"] / num_entailles_2016
df_pr["2011_percent_entailles"] = df_pr["2011_num_entailles"] / num_entailles_2011

df_pr[[
    "2021_percent_entailles",
    "2016_percent_entailles",
    "2011_percent_entailles"
]]

#### Proportions d'exploitations par province 

<span style="color:red"> 
La proportion d'exploitations québécoises est plus faible que sa proportion d'entailles. Il y a donc un plus grand nombre moyen d'entailles par exploitation. Le Québec compte tout de même entre 68 et 75 % des exploitations du pays, suivis de l'Ontario avec entre 21 et 26 % des exploitations.
</span>

In [ ]:
df_pr = df.groupby(
    "province"
).sum()

df_pr["2021_percent_exploitations"] = df_pr["2021_num_exploitations"] / num_exploitations_2021
df_pr["2016_percent_exploitations"] = df_pr["2016_num_exploitations"] / num_exploitations_2016
df_pr["2011_percent_exploitations"] = df_pr["2011_num_exploitations"] / num_exploitations_2011

df_pr[[
    "2021_percent_exploitations", 
    "2016_percent_exploitations", 
    "2011_percent_exploitations", 
]]

### 4.2.2. Quelles régions ont connu la plus forte croissance au fil des ans?

On calcule ici les différences de nombre d'entailles entre 2021 et 2011 (`diff_num_entailles`). 

<span style="color:red"> 
Si le Manitoba a connu la plus forte hausse relative, le nombre absolu d'entailles demeure extrêmement bas. Le Nouveau-Brunswick, l'Ontario et la Nouvelle-Écosse ont connu des croissances plus marquées que les régions du Québec.
</span>

In [ ]:
df["diff_num_entailles"] = (df["2021_num_entailles"] - df["2011_num_entailles"]) / df["2021_num_entailles"]

df.sort_values(
    "diff_num_entailles", 
    ascending=False
).reset_index()[[
    "area_name", 
    "province", 
    "diff_num_entailles",
    "2021_num_entailles", 
    "2011_num_entailles", 
    "2021_num_exploitations",
    "2011_num_exploitations", 
    "rank_entailles",
    "rank_exploitations"
]]

## 4.3. Croiser les données sur la répartition et l'abondance de l’érable à sucre du Canada, et les projections de niche climatique d’ici 2100, avec les données sur les entailles d’érable du recensement de l’agriculture 2021.

### 4.3.1. Quelle proportion (et quelles régions) des entailles actuelles se retrouverait dans de nouvelles conditions de croissance?



#### Scénario A (2041-2070)

Ici, on suppose que les entailles se retrouvent exclusivement dans des zones de croissance idéale. On affiche les régions agricoles de recensement qui ne se retrouveraient plus dans cette zone.

<span style="color:red"> 
Le climat des régions agricoles de recensement 2 et 3 de la Nouvelle-Écosse ne serait plus idéal au développement de l'érable à sucre dès 2041-2070. Ces régions comptent près de 400 000 entailles et 86 exploitations acéricoles en 2021.
<br/>
<br/>
On dénombre également 16 exploitations avec moins de 3000 entailles dans les régions agricoles de recensement 2, 5 et 7 du Manitoba qui se retrouveraient à l’extérieur de la niche climatique de l’érable.
</span>

In [ ]:
# Scénario A (2041)
df["2041_A_is_propice"] = df["core_2011_and_2041"] > 0
is_not_propice_2041_A_df = df[df["2041_A_is_propice"] == False].sort_values(
    "2021_num_entailles", 
    ascending=False
).reset_index()

is_not_propice_2041_A_df[[
    "area_name", 
    "province", 
    "2021_num_entailles", 
    "2021_num_exploitations", 
    "core_2011-2040", 
    "rank_entailles",
    "rank_exploitations"
]]

#### Scénario A (2071-2100)

On affiche les régions agricoles de recensement qui se retrouverait plus dans des conditions climatiques idéales.

<span style="color:red"> 
En 2071-2100, huit des dix régions comptant le plus grand nombre d'entailles en 2021 seraient exclues des conditions de croissance idéale pour l'érable. On note Chaudière-Appalaches, l'Estrie, le Bas-Saint-Laurent, le Centre-du-Québec et la Montérégie (Québec) ainsi que les régions agricoles de recensement 1 et 4 au Nouveau-Brunswick et la région de l'Ouest de l'Ontario.
</span>

In [ ]:
# Scénario A (2071)
df["2071_A_is_propice"] = df["core_2011_and_2071"] > 0
is_not_propice_2071_A_df = df[df["2071_A_is_propice"] == False].sort_values(
    "2021_num_entailles", 
    ascending=False
).reset_index()

is_not_propice_2071_A_df[[
    "area_name", 
    "province", 
    "2021_num_entailles", 
    "2021_num_exploitations", 
    "rank_entailles",
    "core_2011-2040",
    "rank_exploitations"
]]

#### Scénario B (2041-2070)

On affiche les régions agricoles de recensement qui se retrouverait plus dans des conditions climatiques idéales ou tolérables.

<span style="color:red"> 
Selon le scénario B, aucune région importante au développement des produits de l'érable ne se retrouverait totalement exclue de la niche climatique de l’érable en 2041-2070.
</span>

In [ ]:
# Scénario B (2041)
df["2041_B_is_propice"] = df["marginal_2011_and_2041"] > 0
is_not_propice_2041_B_df = df[df["2041_B_is_propice"] == False].sort_values(
    "2021_num_entailles", 
    ascending=False
).reset_index()

is_not_propice_2041_B_df[[
    "area_name", 
    "province", 
    "2021_num_entailles", 
    "2021_num_exploitations", 
    "core_marginal_2011-2040",
    "rank_entailles",
    "rank_exploitations"
]]

#### Scénario B (2071-2100)

On affiche les régions agricoles de recensement qui ne se retrouvent plus dans la niche climatique.

<span style="color:red"> 
Il en va de même en 2071-2100 : aucune région importante au développement des produits de l'érable ne se trouverait totalement exclue de la zone idéale ou tolérable.
</span>

In [ ]:
# Scénario B (2071)
df["2071_B_is_propice"] = df["marginal_2011_and_2071"] > 0
is_not_propice_2071_B_df = df[df["2071_B_is_propice"] == False].sort_values(
    "2021_num_entailles", 
    ascending=False
).reset_index()

is_not_propice_2071_B_df[[
    "area_name", 
    "province", 
    "2021_num_entailles", 
    "2021_num_exploitations", 
    "core_marginal_2011-2040",
    "rank_entailles",
    "rank_exploitations"
]]

#### Proportions d'entailles et d'exploitations ne se retrouvant plus dans les zones propices selon les scénarios A et B et pour les deux périodes de projection.

<span style="color:red"> 
Au total, selon le scénario A, c'est 91,2 % des entailles et 90,4 % des exploitations acéricoles qui se retrouveraient à l'extérieur de la niche climatique idéale en 2071-2100.
    
Les proportions sont minimes pour le scénario A en 2041-2070, ainsi que pour le scénario B. 
</span>

In [ ]:
ratio_entailles_A_2041 = is_not_propice_2041_A_df["2021_num_entailles"].sum() / df["2021_num_entailles"].sum()
ratio_entailles_A_2071 = is_not_propice_2071_A_df["2021_num_entailles"].sum() / df["2021_num_entailles"].sum()

ratio_entailles_B_2041 = is_not_propice_2041_B_df["2021_num_entailles"].sum() / df["2021_num_entailles"].sum()
ratio_entailles_B_2071 = is_not_propice_2071_B_df["2021_num_entailles"].sum() / df["2021_num_entailles"].sum()

ratio_exploitations_A_2041 = is_not_propice_2041_A_df["2021_num_exploitations"].sum() / df["2021_num_exploitations"].sum()
ratio_exploitations_A_2071 = is_not_propice_2071_A_df["2021_num_exploitations"].sum() / df["2021_num_exploitations"].sum()

ratio_exploitations_B_2041 = is_not_propice_2041_B_df["2021_num_exploitations"].sum() / df["2021_num_exploitations"].sum()
ratio_exploitations_B_2071 = is_not_propice_2071_B_df["2021_num_exploitations"].sum() / df["2021_num_exploitations"].sum()


print(f"Scénario A (entailles) : {round(ratio_entailles_A_2041, 3)} (2041), {round(ratio_entailles_A_2071, 3)} (2071)")
print(f"Scénario A (exploitations) : {round(ratio_exploitations_A_2041, 3)} (2041), {round(ratio_exploitations_A_2071, 3)} (2071)")
print(f"Scénario B (entailles) : {round(ratio_entailles_B_2041, 3)} (2041), {round(ratio_entailles_B_2071, 3)} (2071)")
print(f"Scénario B (exploitations) : {round(ratio_exploitations_B_2041, 3)} (2041), {round(ratio_exploitations_B_2071, 3)} (2071)")

### 4.3.2. À quel point les régions qui ont connu la plus forte croissance resteront-elles dans des zones propices au développement de l’érable ou se retrouveront-elles au contraire dans les zones où sa croissance pourrait être affectée?

On affiche les données ordonnées selon la perte d'entailles entre 2021 et 2011 (`diff_num_entailles`). 
- On indique également si les régions se retrouvent dans la zone propice (`YYYY_X_is_propice`) selon l'année de début de la projection (`YYYY`) et le scénario (`X`). 

<span style="color:red"> 
Le climat des sept régions ayant connu la plus forte croissance du nombre d'entailles de 2011 à 2021 ne serait plus idéal au développement de l'érable en 2071-2100.
</span>

In [ ]:
df.sort_values("diff_num_entailles", ascending=False).reset_index()[[
    "area_name",
    "province",
    "diff_num_entailles",
    "2041_A_is_propice",
    "2071_A_is_propice",
    "2041_B_is_propice",
    "2071_B_is_propice"
]]

# 5. Autres questions

## 5.1. Proportion des entailles / exploitations dans des zones qui ne sont pas propices?

On constate que moins de 1% des entailles totales se retrouvent à l'extérieur de la niche climatique core.

In [ ]:
df["2021_percent_exploitations"] = df["2021_num_exploitations"] / num_exploitations_2021

df[
    df["core_2011-2040"] == 0
].sort_values(
    "2021_percent_exploitations", 
    ascending=False
).reset_index()[[
    "2021_percent_exploitations", 
    "area_name"
]]

On constate que moins de 1% des entailles totales se retrouvent à l'extérieur de la niche climatique marginale.

In [ ]:
df["2021_percent_exploitations"] = df["2021_num_exploitations"] / num_exploitations_2021

df[
    df["core_marginal_2011-2040"] == 0
].sort_values(
    "2021_percent_exploitations", 
    ascending=False
).reset_index()[[
    "2021_percent_exploitations", 
    "area_name"
]]